In [3]:
import numpy as np
import tensorflow as tf

import scipy
import os
from os import listdir
from scipy.io import loadmat
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split 

In [4]:
from tensorflow.python.client import device_lib
from tensorflow.compat.v1.keras.backend import set_session

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU

config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.compat.v1.Session(config=config)

set_session(sess)
print(device_lib.list_local_devices())


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4046803580047596474
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12958402281358943626
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 571743436001025966
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7009469728
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17402970460495430772
physical_device_desc: "device: 0, name: GeForce GTX 1070 Ti, pci bus id: 0000:01:00.0, compute capabilit

In [5]:
directory = "./data/eeg_feature_smooth/1/"
directories = ["./data/eeg_feature_smooth/{}/".format(i+1) for i in range(3)] 
print(directories)
channel_coords = [['0', '0', 'AF3', 'FP1', 'FPZ', 'FP2', 'AF4', '0', '0'], ['F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8'], ['FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8'], ['T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8'], ['TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8'], ['P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8'], ['0', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', '0'], ['0', '0', 'CB1', 'O1', 'OZ', 'O2', 'CB2', '0', '0']]

channel_list = ['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2']
print(len(channel_coords), len(channel_coords[0]))
coord_dict = {}
for n in range(len(channel_list)):
    for i, l in enumerate(channel_coords):
        for j, x in enumerate(l):
            if (channel_list[n] == x):
                coord_dict[n] = (i,j)
print(coord_dict)

['./data/eeg_feature_smooth/1/', './data/eeg_feature_smooth/2/', './data/eeg_feature_smooth/3/']
8 9
{0: (0, 3), 1: (0, 4), 2: (0, 5), 3: (0, 2), 4: (0, 6), 5: (1, 0), 6: (1, 1), 7: (1, 2), 8: (1, 3), 9: (1, 4), 10: (1, 5), 11: (1, 6), 12: (1, 7), 13: (1, 8), 14: (2, 0), 15: (2, 1), 16: (2, 2), 17: (2, 3), 18: (2, 4), 19: (2, 5), 20: (2, 6), 21: (2, 7), 22: (2, 8), 23: (3, 0), 24: (3, 1), 25: (3, 2), 26: (3, 3), 27: (3, 4), 28: (3, 5), 29: (3, 6), 30: (3, 7), 31: (3, 8), 32: (4, 0), 33: (4, 1), 34: (4, 2), 35: (4, 3), 36: (4, 4), 37: (4, 5), 38: (4, 6), 39: (4, 7), 40: (4, 8), 41: (5, 0), 42: (5, 1), 43: (5, 2), 44: (5, 3), 45: (5, 4), 46: (5, 5), 47: (5, 6), 48: (5, 7), 49: (5, 8), 50: (6, 1), 51: (6, 2), 52: (6, 3), 53: (6, 4), 54: (6, 5), 55: (6, 6), 56: (6, 7), 57: (7, 2), 58: (7, 3), 59: (7, 4), 60: (7, 5), 61: (7, 6)}


In [6]:
n = 24
perSample = ['de_movingAve', 'de_LDS', 'psd_movingAve', 'psd_LDS']
array = np.zeros(shape=(len(directories),len(os.listdir(directories[0])), n, 4, 8, 9, 5, 64)) # features = 4 datatypes*(8 x 9 eeg channel locs)*5 frequency bands*64 timestamps(zero padded) // trials = (3 sessions) x 15 people x 24 labels 
li = []
for h, dire in enumerate(directories):
    print(dire)
    data = [loadmat(dire + file) for file in os.listdir(dire)]
    for i, bigsample in enumerate(data):
        for j in range(n):
            for k, key in enumerate(perSample):
                sample = np.transpose(np.array(bigsample[key + str(j+1)]), (0,2,1))
                sample = np.pad(sample, [(0,0), (0,0), (0, 64-sample.shape[2])])
                for l, channel in enumerate(sample):
                    array[h][i][j][k][coord_dict[l][0]][coord_dict[l][1]] = channel

print(array.shape)



./data/eeg_feature_smooth/1/
./data/eeg_feature_smooth/2/
./data/eeg_feature_smooth/3/
(3, 15, 24, 4, 8, 9, 5, 64)


In [7]:
_X = array.reshape(np.prod(array.shape[0:3]), *array.shape[3:])
_X.shape

(1080, 4, 8, 9, 5, 64)

In [8]:
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
labels = {0: 'neutral', 1: 'sad', 2: 'fear', 3: 'happy'}

y = np.array(session1_label * 15 + session2_label * 15 + session3_label * 15)
y.shape

(1080,)

In [9]:
def crossval(generate_model, n_epochs, X_train, y_train, X_test, y_test, filename = None):
    kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=0)
    bestmodel = None
    bestAcc = 0
    cvscores = []
    fold = 1
    for train, test in kfold.split(X_train, y_train):
        model = generate_model()
        model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold} ...')
        model.fit(X_train[train], y_train[train],epochs=n_epochs, verbose=0, validation_split=0.2)
        scores = model.evaluate(X_train[test], y_train[test], verbose=1)
        print("Score for fold %d - %s: %.2f%%" % (fold, model.metrics_names[1], scores[1]*100))
        if(scores[1] > bestAcc):
            bestAcc = scores[1]
            bestmodel = model
        cvscores.append(scores[1] * 100)
        fold += 1
    print('------------------------------------------------------------------------')
    print("Avg accuracies: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    test_loss, test_acc = bestmodel.evaluate(X_test,  y_test, verbose=2)
    print('\nTest accuracy:', test_acc)
    if filename:
        pickle.dump( bestmodel, open( filename, "wb" ) )

In [10]:
def fully_conv():
    return tf.keras.Sequential([
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.SeparableConv1D(
128, 3, data_format='channels_last', padding="same", activation='relu'),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [11]:
def single_conv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv1D(
32, 3, activation='relu',input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling1D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4)
])

In [12]:
def LSTM():
    return tf.keras.Sequential([
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
    ])

In [13]:
def dense():
    return tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X.shape[1:]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(4)
])

In [14]:
def twodConv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv2D(
32, 3, padding="same", activation="relu",input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.BatchNormalization(center=True, scale=True),
    tf.keras.layers.Conv2D(
64, 3, padding="same", activation="relu",input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.BatchNormalization(center=True, scale=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [15]:
def threedConv():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv3D(32, kernel_size=10, padding="same", activation='relu', kernel_initializer='he_uniform', input_shape=X.shape[1:]))
    model.add(tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)))
    #model.add(tf.keras.layers.BatchNormalization())
    #model.add(tf.keras.layers.Dropout(0.5))
    #model.add(tf.keras.layers.Conv3D(64, kernel_size=3, activation='relu', padding="same", kernel_initializer='he_uniform'))
    #model.add(tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)))
    #model.add(tf.keras.layers.BatchNormalization(center=True, scale=True))
    #model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(10))
    return model

In [16]:
X = _X.transpose(0, 5, 1,2,3,4)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 4, 8, 9, 5)
(1080, 64, 1440)


In [17]:
crossval(dense, 40, X_train, y_train, X_test, y_test)

------------------------------------------------------------------------
Training for fold 1 ...
7/7 [==============================] - 0s 4ms/step - loss: 132050960.0000 - accuracy: 0.5278
Score for fold 1 - accuracy: 52.78%
------------------------------------------------------------------------
Training for fold 2 ...
7/7 [==============================] - 0s 5ms/step - loss: 306641984.0000 - accuracy: 0.6019
Score for fold 2 - accuracy: 60.19%
------------------------------------------------------------------------
Training for fold 3 ...
7/7 [==============================] - 0s 4ms/step - loss: 900130176.0000 - accuracy: 0.4491
Score for fold 3 - accuracy: 44.91%
------------------------------------------------------------------------
Training for fold 4 ...
7/7 [==============================] - 0s 4ms/step - loss: 210064016.0000 - accuracy: 0.6065
Score for fold 4 - accuracy: 60.65%
------------------------------------------------------------------------
Avg accuracies: 54.63% 

In [18]:
crossval(single_conv, 40, X_train, y_train, X_test, y_test)

------------------------------------------------------------------------
Training for fold 1 ...
7/7 [==============================] - 0s 5ms/step - loss: 24415792.0000 - accuracy: 0.5139
Score for fold 1 - accuracy: 51.39%
------------------------------------------------------------------------
Training for fold 2 ...
7/7 [==============================] - 0s 4ms/step - loss: 287301024.0000 - accuracy: 0.4028
Score for fold 2 - accuracy: 40.28%
------------------------------------------------------------------------
Training for fold 3 ...
7/7 [==============================] - 0s 4ms/step - loss: 237452944.0000 - accuracy: 0.6620
Score for fold 3 - accuracy: 66.20%
------------------------------------------------------------------------
Training for fold 4 ...
7/7 [==============================] - 0s 4ms/step - loss: 220737200.0000 - accuracy: 0.4769
Score for fold 4 - accuracy: 47.69%
------------------------------------------------------------------------
Avg accuracies: 51.39% (

In [19]:
X = _X.transpose(0, 5, 2,3, 4, 1)[:,:,:,:,:,[0,2]]

X = X.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3], np.prod(X.shape[4:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 8, 9, 10)


In [20]:
crossval(threedConv, 100, X_train, y_train, X_test, y_test)

------------------------------------------------------------------------
Training for fold 1 ...
7/7 [==============================] - 0s 39ms/step - loss: 394562272.0000 - accuracy: 0.5787
Score for fold 1 - accuracy: 57.87%
------------------------------------------------------------------------
Training for fold 2 ...
7/7 [==============================] - 0s 25ms/step - loss: 104632808.0000 - accuracy: 0.4861
Score for fold 2 - accuracy: 48.61%
------------------------------------------------------------------------
Training for fold 3 ...
7/7 [==============================] - 0s 25ms/step - loss: 5871898112.0000 - accuracy: 0.4676
Score for fold 3 - accuracy: 46.76%
------------------------------------------------------------------------
Training for fold 4 ...
7/7 [==============================] - 0s 25ms/step - loss: 340366336.0000 - accuracy: 0.5278
Score for fold 4 - accuracy: 52.78%
------------------------------------------------------------------------
Avg accuracies: 51